# NIFTY50 Stock Summary (Table 1)

### 🎯 Objective
In this notebook, we will:
1. Scrape **NIFTY50 stock summary** from Yahoo Finance.
2. Save the **raw dataset** in `/data/raw/`.
3. Clean and standardize the dataset:
   - Normalize column names
   - Convert numeric columns
   - Handle missing values
4. Save the **cleaned dataset** in `/data/cleaned/`.


----
## Starting with Phase 1
----

## Step 1: Setup Directories and Import Libraries

We begin by importing Python libraries and creating dedicated directories to keep our raw data and cleaned data organized.

In [1]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import os

# Create directories
os.makedirs("../data/raw", exist_ok=True)
os.makedirs("../data/cleaned", exist_ok=True)


## Step 2: Retrieve Raw Data from NSE API

The NSE India API provides structured JSON data of NIFTY 50 constituents.
We send a request with a browser-like User-Agent header to prevent the server from blocking our request.

URL: https://www.nseindia.com/market-data/live-equity-market?symbol=NIFTY%2050


In [30]:
print("Status:", response.status_code)
print("Content-Type:", response.headers.get("Content-Type"))
print("First 500 chars:\n", response.text[:500])


Status: 200
Content-Type: application/json; charset=utf-8
First 500 chars:
 {"name":"NIFTY 50","advance":{"declines":"22","advances":"28","unchanged":"0"},"timestamp":"05-Sep-2025 16:00:00","data":[{"priority":1,"symbol":"NIFTY 50","identifier":"NIFTY 50","open":24818.85,"dayHigh":24832.35,"dayLow":24621.6,"lastPrice":24741,"previousClose":24734.3,"change":6.700000000000728,"pChange":0.03,"ffmc":1119690269.13,"yearHigh":26277.35,"yearLow":21743.65,"totalTradedVolume":210071924,"stockIndClosePrice":0,"totalTradedValue":185301526994.18,"lastUpdateTime":"05-Sep-2025 16:00:


In [10]:
import requests
url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050"
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure the request was successful
data_json = response.json()


## Step 3: Save Raw JSON for Reference

It’s good practice to store the raw unmodified data for traceability.
This ensures we always have the original source if something goes wrong in cleaning.

In [11]:
with open("../data/raw/stock_summary_raw.json", "w") as f:
    f.write(response.text)


## Step 4: Parse JSON into a DataFrame

The raw JSON contains a key "data" where all stock information is stored.
We will extract it and convert it into a structured pandas DataFrame.

In [12]:
records = data_json.get("data", [])
df_raw = pd.DataFrame(records)

print("Raw columns:", df_raw.columns)
df_raw.head()


Raw columns: Index(['priority', 'symbol', 'identifier', 'open', 'dayHigh', 'dayLow',
       'lastPrice', 'previousClose', 'change', 'pChange', 'ffmc', 'yearHigh',
       'yearLow', 'totalTradedVolume', 'stockIndClosePrice',
       'totalTradedValue', 'lastUpdateTime', 'nearWKH', 'nearWKL',
       'perChange365d', 'date365dAgo', 'chart365dPath', 'date30dAgo',
       'perChange30d', 'chart30dPath', 'chartTodayPath', 'series', 'meta'],
      dtype='object')


,priority,symbol,identifier,open,dayHigh,dayLow,lastPrice,previousClose,change,pChange,...,nearWKL,perChange365d,date365dAgo,chart365dPath,date30dAgo,perChange30d,chart30dPath,chartTodayPath,series,meta
0,1,NIFTY 50,NIFTY 50,24818.85,24832.35,24621.6,24741.00,24734.3,6.70,0.03,...,-13.784944,-1.61,05-Sep-2024,https://nsearchives.nseindia.com/365d/NIFTY-50...,06-Aug-2025,0.68,https://nsearchives.nseindia.com/30d/NIFTY-50.svg,https://nsearchives.nseindia.com/today/NIFTY-5...,NaN,NaN
1,0,EICHERMOT,EICHERMOTEQN,6449.00,6598.50,6440.5,6580.00,6425.0,155.00,2.41,...,-45.938453,37.09,05-Sep-2024,https://nsearchives.nseindia.com/365d/EICHERMO...,06-Aug-2025,16.10,https://nsearchives.nseindia.com/30d/EICHERMOT...,https://nsearchives.nseindia.com/today/EICHERM...,EQ,"{'symbol': 'EICHERMOT', 'companyName': 'Eicher..."
2,0,M&M,M&MEQN,3511.00,3577.80,3481.5,3562.90,3481.5,81.40,2.34,...,-46.923711,30.78,05-Sep-2024,https://nsearchives.nseindia.com/365d/M&M-EQ.svg,06-Aug-2025,10.34,https://nsearchives.nseindia.com/30d/M&M-EQ.svg,https://nsearchives.nseindia.com/today/M&MEQN.svg,EQ,"{'symbol': 'M&M', 'companyName': 'Mahindra & M..."
3,0,SHRIRAMFIN,SHRIRAMFINEQN,589.00,596.00,585.6,595.65,585.8,9.85,1.68,...,-20.735786,-81.74,05-Sep-2024,https://nsearchives.nseindia.com/365d/SHRIRAMF...,06-Aug-2025,-6.04,https://nsearchives.nseindia.com/30d/SHRIRAMFI...,https://nsearchives.nseindia.com/today/SHRIRAM...,EQ,"{'symbol': 'SHRIRAMFIN', 'companyName': 'Shrir..."
4,0,MARUTI,MARUTIEQN,14740.00,14957.00,14701.0,14903.00,14662.0,241.00,1.64,...,-38.955711,21.11,05-Sep-2024,https://nsearchives.nseindia.com/365d/MARUTI-E...,06-Aug-2025,18.93,https://nsearchives.nseindia.com/30d/MARUTI-EQ...,https://nsearchives.nseindia.com/today/MARUTIE...,EQ,"{'symbol': 'MARUTI', 'companyName': 'Maruti Su..."


## Step 5: Save Raw CSV Snapshot

Along with JSON, we save the raw CSV to make it easier for analysts who prefer working directly with tabular data

In [13]:
df_raw.to_csv("../data/raw/stock_summary_raw.csv", index=False)


## Step 6: Clean and Standardize Column Names

Raw column names may contain spaces, mixed cases, and special characters.
We will normalize column names to lowercase, snake_case, and align them with our project schema.

In [14]:
df = df_raw.copy()

# Normalize column names
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^a-z0-9_]", "", regex=True)
)

# Rename selected fields to standard names
rename_map = {
    "symbol": "symbol",
    "lastprice": "last_price",
    "change": "change",
    "pchange": "perc_change",
    "open": "open",
    "totaltradedvolume": "volume",
    "totaltradedvalue": "value",
    
    # Additional mappings
    "dayhigh": "high",
    "daylow": "low",
    "previousclose": "prev_close",
    "ffmc": "ffmc",
    "yearhigh": "year_high",
    "yearlow": "year_low",
    "stockindcloseprice": "index_close_price",
    "lastupdatetime": "last_update",
    "nearwkh": "near_week_high",
    "nearwkl": "near_week_low",
    "perchange365d": "perc_change_365d",
    "date365dago": "date_365d_ago",
    "chart365dpath": "chart_365d_path",
    "date30dago": "date_30d_ago",
    "perchange30d": "perc_change_30d",
    "chart30dpath": "chart_30d_path",
    "charttodaypath": "chart_today_path",
    "series": "series",
    "meta": "meta"

}
df = df.rename(columns=rename_map)

df.head()


,priority,symbol,identifier,open,high,low,last_price,prev_close,change,perc_change,...,near_week_low,perc_change_365d,date_365d_ago,chart_365d_path,date_30d_ago,perc_change_30d,chart_30d_path,chart_today_path,series,meta
0,1,NIFTY 50,NIFTY 50,24818.85,24832.35,24621.6,24741.00,24734.3,6.70,0.03,...,-13.784944,-1.61,05-Sep-2024,https://nsearchives.nseindia.com/365d/NIFTY-50...,06-Aug-2025,0.68,https://nsearchives.nseindia.com/30d/NIFTY-50.svg,https://nsearchives.nseindia.com/today/NIFTY-5...,NaN,NaN
1,0,EICHERMOT,EICHERMOTEQN,6449.00,6598.50,6440.5,6580.00,6425.0,155.00,2.41,...,-45.938453,37.09,05-Sep-2024,https://nsearchives.nseindia.com/365d/EICHERMO...,06-Aug-2025,16.10,https://nsearchives.nseindia.com/30d/EICHERMOT...,https://nsearchives.nseindia.com/today/EICHERM...,EQ,"{'symbol': 'EICHERMOT', 'companyName': 'Eicher..."
2,0,M&M,M&MEQN,3511.00,3577.80,3481.5,3562.90,3481.5,81.40,2.34,...,-46.923711,30.78,05-Sep-2024,https://nsearchives.nseindia.com/365d/M&M-EQ.svg,06-Aug-2025,10.34,https://nsearchives.nseindia.com/30d/M&M-EQ.svg,https://nsearchives.nseindia.com/today/M&MEQN.svg,EQ,"{'symbol': 'M&M', 'companyName': 'Mahindra & M..."
3,0,SHRIRAMFIN,SHRIRAMFINEQN,589.00,596.00,585.6,595.65,585.8,9.85,1.68,...,-20.735786,-81.74,05-Sep-2024,https://nsearchives.nseindia.com/365d/SHRIRAMF...,06-Aug-2025,-6.04,https://nsearchives.nseindia.com/30d/SHRIRAMFI...,https://nsearchives.nseindia.com/today/SHRIRAM...,EQ,"{'symbol': 'SHRIRAMFIN', 'companyName': 'Shrir..."
4,0,MARUTI,MARUTIEQN,14740.00,14957.00,14701.0,14903.00,14662.0,241.00,1.64,...,-38.955711,21.11,05-Sep-2024,https://nsearchives.nseindia.com/365d/MARUTI-E...,06-Aug-2025,18.93,https://nsearchives.nseindia.com/30d/MARUTI-EQ...,https://nsearchives.nseindia.com/today/MARUTIE...,EQ,"{'symbol': 'MARUTI', 'companyName': 'Maruti Su..."


## Step 7: Convert Numerical Columns

By default, some fields may be stored as strings.
We will explicitly convert them into numeric values for proper analysis.

In [15]:
numeric_cols = ["last_price", "change", "perc_change", "open", "high", "low", "volume", "value"]

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")


## Step 8: Handle Missing Values

To ensure data quality, we will handle missing values by:

Dropping rows where the stock symbol is missing.

Filling missing numerical fields with the median value of that column.

In [16]:
# Drop invalid rows
df = df.dropna(subset=["symbol"])

# Fill missing numeric fields
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())


### Further cleaning and standardising 

* Check the column headers to determine if further standardisation is required.
* Check for more missing values and replace them with a default  or a  suitable value.
* Check the datatypes of each date & time-specific column. If it's a string, convert it to datetime format.

In [17]:
#Check the column headers
print(df.columns)


Index(['priority', 'symbol', 'identifier', 'open', 'high', 'low', 'last_price',
       'prev_close', 'change', 'perc_change', 'ffmc', 'year_high', 'year_low',
       'volume', 'index_close_price', 'value', 'last_update', 'near_week_high',
       'near_week_low', 'perc_change_365d', 'date_365d_ago', 'chart_365d_path',
       'date_30d_ago', 'perc_change_30d', 'chart_30d_path', 'chart_today_path',
       'series', 'meta'],
      dtype='object')


In [18]:
#print(df.loc[8, 'perc_change_365d']) => Null
df.loc[8, 'perc_change_365d'] = 0
print(df.loc[8, 'perc_change_365d'])


0


In [19]:
print(df.loc[8, 'date_365d_ago'])
df.loc[8, 'date_365d_ago'] = '05-Sep-2024'
print(df.loc[8, 'date_365d_ago'])

-
05-Sep-2024


In [20]:
df.loc[8, 'chart_365d_path'] = 'Missing.svg'
print(df.loc[8, 'chart_365d_path'])

Missing.svg


#### Check for the datatype of date or time-specific columns 

In [21]:
print(type('last_update'))

<class 'str'>


In [22]:
print(type('date_30d_ago'))

<class 'str'>


#### Before conversion make sure all missing values are dealt with 

In [23]:
# Get the first non-null value from last_update
last_update_value = df["last_update"].dropna().iloc[0]

# Fill all missing values with this timestamp
df["last_update"] = df["last_update"].fillna(last_update_value)

# Quick check
df["last_update"].unique()


array(['05-Sep-2025 16:00:00'], dtype=object)

In [24]:
print(df.loc[8, 'last_update'])

05-Sep-2025 16:00:00


#### After dealing with the missing values now convert their data types

In [25]:
# Define date columns
date_cols = ["last_update", "date_365d_ago", "date_30d_ago"]

# Convert string → datetime
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=False)

# Verify conversions
df.dtypes[["last_update", "date_365d_ago", "date_30d_ago"]]


last_update      datetime64[ns]
date_365d_ago    datetime64[ns]
date_30d_ago     datetime64[ns]
dtype: object

## Step 9: Save the Final Cleaned Dataset

Finally, we save the cleaned dataset which is ready for analysis or visualization.

In [26]:
df.to_csv("../data/cleaned/indstock_summary_cleaned.csv", index=False)
print("✅ Cleaned dataset saved successfully!")


✅ Cleaned dataset saved successfully!


----
## Starting with Phase 2
----

## Step 10: Now Extract The Meta Column

We will save the `meta` column dataset into `../data/raw`, naming it `indstock_metadata.csv`.

In [27]:
import pandas as pd
import ast

# ✅ Step 1: Load your cleaned stock summary file
df = pd.read_csv("../data/cleaned/indstock_summary_cleaned.csv")

# ✅ Step 2: Drop rows where 'meta' is empty
df_meta = df.dropna(subset=["meta"]).copy()

# ✅ Step 3: Convert 'meta' string into Python dictionary
df_meta["meta_dict"] = df_meta["meta"].apply(lambda x: ast.literal_eval(x))

# ✅ Step 4: Expand the dictionary into separate columns
meta_expanded = pd.json_normalize(df_meta["meta_dict"])

# ✅ Step 5: Keep symbol + metadata together
meta_final = pd.concat([df_meta[["symbol"]].reset_index(drop=True),
                        meta_expanded.reset_index(drop=True)], axis=1)

# ✅ Step 6: Save as new CSV file
meta_final.to_csv("../data/raw/indstock_metadata.csv", index=False)

print("Metadata extracted and saved to 'indstock_metadata.csv'")
print(meta_final.head())


Metadata extracted and saved to 'indstock_metadata.csv'
       symbol      symbol                       companyName  \
0   EICHERMOT   EICHERMOT             Eicher Motors Limited   
1         M&M         M&M       Mahindra & Mahindra Limited   
2  SHRIRAMFIN  SHRIRAMFIN           Shriram Finance Limited   
3      MARUTI      MARUTI       Maruti Suzuki India Limited   
4     DRREDDY     DRREDDY  Dr. Reddy's Laboratories Limited   

                               industry activeSeries debtSeries  isFNOSec  \
0                          2/3 Wheelers         [EQ]         []      True   
1     Passenger Cars & Utility Vehicles     [EQ, T0]         []      True   
2  Non Banking Financial Company (NBFC)         [EQ]         []      True   
3     Passenger Cars & Utility Vehicles         [EQ]         []      True   
4                       Pharmaceuticals         [EQ]         []      True   

   isCASec  isSLBSec  isDebtSec  ...  isETFSec isDelisted          isin  \
0    False      True      F

####  Check for the Column heads

In [28]:
meta_final.columns


Index(['symbol', 'symbol', 'companyName', 'industry', 'activeSeries',
       'debtSeries', 'isFNOSec', 'isCASec', 'isSLBSec', 'isDebtSec',
       'isSuspended', 'tempSuspendedSeries', 'isETFSec', 'isDelisted', 'isin',
       'slb_isin', 'listingDate', 'isMunicipalBond', 'isHybridSymbol',
       'quotepreopenstatus.equityTime', 'quotepreopenstatus.preOpenTime',
       'quotepreopenstatus.QuotePreOpenFlag'],
      dtype='object')

## Step 11: Standardisation of Column Header Name 

Rename column header into a uniform pattern.

In [29]:
import pandas as pd

# Load file
df_meta = pd.read_csv("../data/raw/indstock_metadata.csv")

# Define rename mapping
rename_map = {
    "companyName": "company_name",
    "industry": "industry",
    "activeSeries": "active_series",
    "debtSeries": "debt_series",
    "isFNOSec": "is_fno_sec",
    "isCASec": "is_ca_sec",
    "isSLBSec": "is_slb_sec",
    "isDebtSec": "is_debt_sec",
    "isSuspended": "is_suspended",
    "tempSuspendedSeries": "temp_suspended_series",
    "isETFSec": "is_etf_sec",
    "isDelisted": "is_delisted",
    "isin": "isin",
    "slb_isin": "slb_isin",
    "listingDate": "listing_date",
    "isMunicipalBond": "is_municipal_bond",
    "isHybridSymbol": "is_hybrid_symbol",
    "quotepreopenstatus.equityTime": "equity_time",
    "quotepreopenstatus.preOpenTime": "pre_open_time",
    "quotepreopenstatus.QuotePreOpenFlag": "quote_pre_open_flag",
}

# Apply rename
df_meta = df_meta.rename(columns=rename_map)

# Remove duplicate "symbol" column if needed
df_meta = df_meta.loc[:, ~df_meta.columns.duplicated()]

# Save cleaned file
df_meta.to_csv("../data/cleaned/indstock_metadata_cleaned.csv", index=False)

print("✅ Standardized metadata saved as 'indstock_metadata_cleaned.csv'")
print(df_meta.head())


✅ Standardized metadata saved as 'indstock_metadata_cleaned.csv'
       symbol    symbol.1                      company_name  \
0   EICHERMOT   EICHERMOT             Eicher Motors Limited   
1         M&M         M&M       Mahindra & Mahindra Limited   
2  SHRIRAMFIN  SHRIRAMFIN           Shriram Finance Limited   
3      MARUTI      MARUTI       Maruti Suzuki India Limited   
4     DRREDDY     DRREDDY  Dr. Reddy's Laboratories Limited   

                               industry active_series debt_series  is_fno_sec  \
0                          2/3 Wheelers        ['EQ']          []        True   
1     Passenger Cars & Utility Vehicles  ['EQ', 'T0']          []        True   
2  Non Banking Financial Company (NBFC)        ['EQ']          []        True   
3     Passenger Cars & Utility Vehicles        ['EQ']          []        True   
4                       Pharmaceuticals        ['EQ']          []        True   

   is_ca_sec  is_slb_sec  is_debt_sec  ...  is_etf_sec is_delisted  \

##


***

## 📊 Conclusion

We have successfully:
* Extracted the complete stock summary of all NIFTY 50 constituents.
* Organized the workflow into raw and cleaned datasets.
* Ensured numerical consistency and handled missing values.

This cleaned dataset can now be directly used for:
* **Exploratory Data Analysis (EDA)**
* **Visualization Dashboards**
* **Integration into larger financial projects**

---